### 01 packages

In [11]:
##################################################################################################
##################################################################################################

import numpy as np

import pandas as pd

import geopandas as gpd

import folium

import copy

import geopandas as gp

import networkx as nx

from sklearn.cluster import DBSCAN

from collections import defaultdict

from shapely.geometry import Point

from scipy.spatial import KDTree

from datetime import datetime, timedelta

import os

import matplotlib.pyplot as plt

import json

##################################################################################################
##################################################################################################

import warnings

warnings.filterwarnings('ignore')

### 02 suburb boundary

https://actmapi-actgov.opendata.arcgis.com/search


The ACT Divisions data sets depict the authoritative boundaries and names of suburbs for addressing and location purposes.

In [5]:
import sys

In [7]:
sys.executable

'/opt/anaconda3/bin/python'

In [9]:
!/opt/anaconda3/bin/python -m pip install folium

In [13]:
##################################################################################################
##################################################################################################

suburb_path="./01data/canberra_suburb"

suburb_data = gp.read_file(suburb_path+"/ACTGOV_DIVISION.shp")

suburb_data = suburb_data.to_crs(epsg=4326)

##################################################################################################
##################################################################################################

suburbs_with_intersections = gpd.sjoin(suburb_data, suburb_data, how='inner', predicate='intersects')

non_isolated_suburbs = suburbs_with_intersections[suburbs_with_intersections.ID_left != suburbs_with_intersections.ID_right]

non_isolated_suburb_ids = non_isolated_suburbs['ID_left'].unique()

non_isolated_suburbs = suburb_data.loc[suburb_data.ID.isin(non_isolated_suburb_ids)]

##################################################################################################
##################################################################################################

entire_convex_hull = non_isolated_suburbs.unary_union.convex_hull

##################################################################################################
##################################################################################################

network = folium.Map(location=[suburb_data.geometry.centroid.y.mean(), suburb_data.geometry.centroid.x.mean()], zoom_start=11,tiles='CartoDB positron')

folium.Choropleth(
    geo_data=non_isolated_suburbs[['geometry']],
    fill_color='blue',
    fill_opacity=0.1,
    name='Zone').add_to(network)

network.save('./04html/01suburb_map.html')

non_isolated_suburbs.to_file("./02result/road_network/suburb.json", driver="GeoJSON")

network

### 03 wild road network

https://actmapi-actgov.opendata.arcgis.com/search?collection=dataset&q=road

The position and extents of the edges of the air/light rail/rail/road or water carriageway that form the physical boundary between the pavement and the verge of a feature.


In [28]:
##################################################################################################
##################################################################################################

road_path="./01data/canberra_road"

road_data = gp.read_file(road_path+"/ACTGOV_ROAD_EDGES.shp")

road_data = road_data.to_crs(epsg=4326)

##################################################################################################
##################################################################################################

road_data=road_data.loc[(road_data.ROAD_ID!=0)]

road_data['road_type']=road_data.apply(lambda x:x.geometry.type,axis=1)

road_data=road_data.loc[road_data.road_type=="LineString"]

##################################################################################################
##################################################################################################

entire_convex_hull = gpd.GeoSeries([entire_convex_hull])

entire_convex_hull_gdf = gp.GeoDataFrame(geometry=entire_convex_hull, crs=4326)

roads_in_suburbs = gp.sjoin(road_data, entire_convex_hull_gdf, how='inner', predicate='within')

##################################################################################################
##################################################################################################

# folium.GeoJson(roads_in_suburbs).add_to(network)

# network.save('./04html/02road_network_map.html')

### 04 abstracted road network

In [29]:
##################################################################################################
##################################################################################################

graph_road_data=copy.copy(roads_in_suburbs[["ID","DIVISION_N","geometry"]])



graph_road_data['start']=graph_road_data.apply(lambda x:tuple(list(x['geometry'].coords)[0][::-1]),axis=1)

graph_road_data['end']=graph_road_data.apply(lambda x:tuple(list(x['geometry'].coords)[-1][::-1]),axis=1)

##################################################################################################
##################################################################################################

wild_node_set=list(graph_road_data.start.unique())+list(graph_road_data.end.unique())

wild_node_set=list(set(wild_node_set))

np.save('./02result/road_network/wild_node_set',wild_node_set)

##################################################################################################
##################################################################################################

distance_threshold = 0.0005  # Adjust the threshold distance as needed

db = DBSCAN(eps=distance_threshold, min_samples=1).fit(np.array(wild_node_set))

clusters = db.labels_

# Create a mapping of clusters to their points
cluster_dict = defaultdict(list)
node_cluster_dic = {}
for i, cluster_id in enumerate(clusters):
    cluster_dict[cluster_id].append(wild_node_set[i])
    node_cluster_dic[wild_node_set[i]]=cluster_id
    
# Calculate the centroid of each cluster
cluster_centroids = {cluster_id: tuple(np.array(points).mean(axis=0)) for cluster_id, points in cluster_dict.items()}

##################################################################################################
##################################################################################################

graph_road_data['start_cluster']=graph_road_data.apply(lambda x:node_cluster_dic[x.start],axis=1)

graph_road_data['end_cluster']=graph_road_data.apply(lambda x:node_cluster_dic[x.end],axis=1)

graph_road_data=graph_road_data.loc[graph_road_data.start_cluster!=graph_road_data.end_cluster]

graph_road_data=graph_road_data.reset_index(drop=True)

graph_road_data['start_cluster_node']=graph_road_data.apply(lambda x:cluster_centroids[x.start_cluster],axis=1)

graph_road_data['end_cluster_node']=graph_road_data.apply(lambda x:cluster_centroids[x.end_cluster],axis=1)

graph_road_data=graph_road_data[["ID","DIVISION_N","start_cluster_node","end_cluster_node"]]
 

##################################################################################################
##################################################################################################

G = nx.Graph()

for idx,row in graph_road_data.iterrows():
    
    edge_distance=108000*Point(row.start_cluster_node).distance(Point(row.end_cluster_node))
    
    G.add_edge(row.start_cluster_node,row.end_cluster_node, weight=edge_distance)

##################################################################################################
##################################################################################################

# Find all connected components
connected_components = list(nx.connected_components(G))

# Find the largest connected component
largest_component = max(connected_components, key=len)

# Create a subgraph of the largest connected component
largest_subgraph = G.subgraph(largest_component)

# Calculate the number of nodes and edges in the largest connected component
num_nodes = largest_subgraph.number_of_nodes()

print(f"The ratio of nodes covered in the largest connected component is: {num_nodes/G.number_of_nodes()}")

abstracted_node_set=list(largest_subgraph.nodes())

np.save('./02result/road_network/abstracted_node_set',abstracted_node_set)

abstracted_edge_set=list(largest_subgraph.edges())

np.save('./02result/road_network/abstracted_edge_set',abstracted_edge_set) 



The ratio of nodes covered in the largest connected component is: 0.9836165666022184


### 05 add bus stops into the network¶

In [30]:
##################################################################################################
##################################################################################################

gtfs_path="./01data/canberra"

bus_stop_data = pd.read_csv(gtfs_path+'/stops.txt')

bus_stop_data = bus_stop_data[["stop_id","stop_name","stop_lat","stop_lon"]]

##################################################################################################
##################################################################################################

bus_stop_nodes=list()

for idx,row in bus_stop_data.iterrows():
    
    node=(row.stop_lat,row.stop_lon)
    
    bus_stop_nodes.append(node)
    
##################################################################################################
##################################################################################################

# Function to calculate Euclidean distance

def euclidean_distance(coord1, coord2):
    
    return np.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

##################################################################################################
##################################################################################################

road_graph_with_bus_stops=largest_subgraph.copy()

for stop_node in bus_stop_nodes:
    
    ##################################################################################################
    
    # Calculate distances from new node to all existing nodes

    distances = [(node, euclidean_distance(stop_node, node)) for node in abstracted_node_set]
    
    # Sort by distance
    
    distances.sort(key=lambda x: x[1])
    
    # Get the two closest existing nodes
    
    closest_node = distances[0][0]
    
    second_closest_node = distances[1][0]
    
    # Remove the edge between the two closest existing nodes if it exists
    if road_graph_with_bus_stops.has_edge(closest_node, second_closest_node):
        
        road_graph_with_bus_stops.remove_edge(closest_node, second_closest_node)
    
    # Add the new node and connect it to the two closest existing nodes
    road_graph_with_bus_stops.add_node(stop_node)
    
    edge_distance=108000*Point(stop_node).distance(Point(closest_node))
    
    road_graph_with_bus_stops.add_edge(stop_node, closest_node, weight=edge_distance)
    
    edge_distance=108000*Point(stop_node).distance(Point(second_closest_node))
    
    road_graph_with_bus_stops.add_edge(stop_node, second_closest_node, weight=edge_distance)

##################################################################################################
##################################################################################################




In [31]:
abstracted_node_with_bus_stops_set=list(road_graph_with_bus_stops.nodes())

np.save('./02result/road_network/abstracted_node_with_bus_stops_set',abstracted_node_with_bus_stops_set)

abstracted_edge_with_bus_stops_set=list(road_graph_with_bus_stops.edges())

np.save('./02result/road_network/abstracted_edge_with_bus_stops_set',abstracted_edge_with_bus_stops_set) 

In [32]:
network = folium.Map(location=[suburb_data.geometry.centroid.y.mean(),\
                               suburb_data.geometry.centroid.x.mean()],\
                     zoom_start=11,tiles='CartoDB positron')

for edge in abstracted_edge_with_bus_stops_set:
    
    folium.PolyLine(edge,\
                    color="black",\
                    weight=1.0,\
                    opacity=0.2).add_to(network)

for node in abstracted_node_set:
    folium.CircleMarker(location=[node[0],node[1]], radius=1, color='red', fill=True).add_to(network)


for node in bus_stop_nodes:
    
    folium.CircleMarker(location=[node[0],node[1]], radius=3, color='green', fill=True).add_to(network)
    
    
network.save('./04html/11road_network_with_bus_stops.html')

### 05 more informal details 

In [33]:
##################################################################################################
##################################################################################################

node_suburb_dic={}

suburb_node_dic={row.ID:list() for idx, row in non_isolated_suburbs.iterrows()}

for node in abstracted_node_with_bus_stops_set:
    
    node_point=Point(list(node)[::-1])
    
    for idx, row in non_isolated_suburbs.iterrows():
        
        if row.geometry.contains(node_point):
            
            node_suburb_dic[node]=row.ID
            
            suburb_node_dic[row.ID].append(node)
            
            break
            
##################################################################################################
##################################################################################################

np.save('./02result/road_network/node_suburb_dic',node_suburb_dic)

np.save('./02result/road_network/suburb_node_dic',suburb_node_dic)


In [34]:
with open("./02result/road_network/node.txt", "w") as file:
    for item in abstracted_node_with_bus_stops_set:
        file.write(f"{item}\n")

with open("./02result/road_network/edge.txt", "w") as file:
    for item in abstracted_edge_with_bus_stops_set:
        file.write(f"{item}\n")


In [35]:
# Save the coordinates to a JSON file
node_file_path = "./02result/road_network/node.json"
with open(node_file_path, "w") as json_file:
    json.dump(abstracted_node_with_bus_stops_set, json_file, indent=4)

# Save the edges to a JSON file
edge_file_path = "./02result/road_network/edge.json"
with open(edge_file_path, "w") as json_file:
    json.dump(abstracted_edge_with_bus_stops_set, json_file, indent=4)